# 한 년도 데이터 크롤링

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url_base = 'https://www.koreabaseball.com/Record/Team/{category}'
category_list = ['Hitter/Basic1.aspx', 'Hitter/Basic2.aspx', 'Pitcher/Basic1.aspx', 'Pitcher/Basic2.aspx',
                 'Defense/Basic.aspx', 'Runner/Basic.aspx']
category_name_list = ['Hitter_1', 'Hitter_2', 'Pitcher_1', 'Pitcher_2', 'Defense', 'Runner']

def year_data_crawling(input_year):
    
    for category in category_list:
        url = url_base.format(category = category)
        req = requests.get(url)
        html = req.text
        soup = BeautifulSoup(html, 'html.parser')
    
        temp_table = soup.find('div', {'class':'record_result'}) #Get Table Info
        col_tag = temp_table.find_all('td')
        col_list = ['Rank', 'Team']
        for col in col_tag:
            try:
                temp_value = col.attrs['data-id']
                if temp_value not in col_list:
                    col_list.append(temp_value)
                else:
                    break
            except:
                pass
        temp_data = pd.DataFrame(columns = col_list)
    
        i = 0
        index = 0
        col_len = len(col_list)
        while True:
            try:
                temp_data.loc[i] = [x.text for x in temp_table.find_all('td')[index:index+col_len]]
                i += 1
                index += col_len
            except:
                break
        
        temp_data.to_csv('Original Datasets/'+str(input_year)+'_'+str(category_name_list[category_list.index(category)])+'_data.csv', index=False, encoding='cp949')

# 전체 년도 데이터 크롤링

In [3]:
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import pandas as pd
import re
import zipfile
import os
import requests
from tqdm import tqdm

In [4]:
driver = webdriver.Chrome('C:/chrome_driver/85/chromedriver.exe')
driver.implicitly_wait(3)

In [5]:
for category in category_list:
    url = url_base.format(category=category)
    driver.get(url)
    driver.maximize_window()
    
    year_list = [x.strip() for x in driver.find_element_by_class_name('select03').text.split('\n') if len(x)>1]

In [6]:
year_list

['2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020']

In [7]:
for year in tqdm(year_list[:]):
    driver.find_element_by_class_name('select03').click()
    xpath_format = '//option[@value="{year}"]'
    driver.find_element_by_xpath(xpath_format.format(year=year)).click()
    year_data_crawling(year)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:29<00:00,  1.48s/it]


In [8]:
driver.close()